# 과업 1: 수상 레저 속 승선 인원 수 탐지

## 개요
이 노트북은 YOLOv11s 사전 학습 모델을 사용하여 수상 레저 이미지에서 보트와 사람을 탐지하고, 각 보트에 탑승한 인원수를 계산합니다.

## 주요 기능
- **객체 탐지**: YOLOv11s 모델로 사람(Person)과 보트(Boat) 탐지
- **탑승 여부 판별**: 사람 바운딩박스 하단부(발 위치)가 보트 바운딩박스 내부에 포함되는지 확인
- **중복 제거**: IoU 기반으로 중복 탐지된 바운딩박스 병합
- **결과 시각화**: 보트별 탑승 인원수를 이미지에 표시

## 입력 데이터
- `data/test/plate_detection/` 폴더의 이미지 파일들
- `model/yolo11s_passenger_counting.pt` - 사전 학습된 YOLOv11s 모델

## 출력
- `results_onboat/` 폴더에 결과 이미지 저장


In [9]:
# 필요한 라이브러리 import
import sys
from pathlib import Path
import torch

# 프로젝트 루트 경로 설정
BASE_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.insert(0, str(BASE_DIR / "src"))

print(f"프로젝트 루트: {BASE_DIR}")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


프로젝트 루트: c:\Users\Brain\Desktop\watercraft-illegal-analytics
PyTorch 버전: 2.5.1+cu121
CUDA 사용 가능: True
GPU: NVIDIA GeForce RTX 5070


## 1. 설정 및 경로 확인

먼저 필요한 경로와 파일이 존재하는지 확인합니다.


In [11]:
# 경로 설정
MODEL_PATH = BASE_DIR / "model" / "yolo11s_passenger_counting.pt"
TEST_PATH = BASE_DIR / "data" / "test" / "plate_detection"

print(f"[설정] 모델 경로: {MODEL_PATH}")
print(f"[설정] 테스트 이미지 경로: {TEST_PATH}")
print()

# 파일 존재 확인
if not MODEL_PATH.exists():
    print(f"[ERROR] 모델 파일이 없습니다: {MODEL_PATH}")
else:
    print(f"[OK] 모델 파일 확인: {MODEL_PATH}")

if not TEST_PATH.exists():
    print(f"[ERROR] 테스트 이미지 폴더가 없습니다: {TEST_PATH}")
else:
    image_count = len(list(TEST_PATH.glob("*.jpg"))) + len(list(TEST_PATH.glob("*.png")))
    print(f"[OK] 테스트 이미지 폴더 확인: {TEST_PATH} ({image_count}개 이미지)")


[설정] 모델 경로: c:\Users\Brain\Desktop\watercraft-illegal-analytics\model\yolo11s_passenger_counting.pt
[설정] 테스트 이미지 경로: c:\Users\Brain\Desktop\watercraft-illegal-analytics\data\test\plate_detection

[OK] 모델 파일 확인: c:\Users\Brain\Desktop\watercraft-illegal-analytics\model\yolo11s_passenger_counting.pt
[OK] 테스트 이미지 폴더 확인: c:\Users\Brain\Desktop\watercraft-illegal-analytics\data\test\plate_detection (28개 이미지)


## 2. 승선 인원수 탐지 실행

`passenger_counting.py` 스크립트를 실행하여 승선 인원수 탐지를 수행합니다.


In [13]:
# passenger_counting.py 스크립트 실행
# 주의: 이 셀은 실행에 시간이 걸릴 수 있습니다 (이미지 수에 따라)

import subprocess
import os

# 작업 디렉토리를 프로젝트 루트로 변경
script_path = BASE_DIR / "src" / "passenger_counting.py"

if script_path.exists():
    print(f"[INFO] 스크립트 실행 시작: {script_path}")
    print("[INFO] 이 과정은 시간이 걸릴 수 있습니다...")
    print()
    
    # Python 스크립트 실행 (오류 메시지도 출력하도록 설정)
    result = subprocess.run(
        [sys.executable, str(script_path)],
        cwd=str(BASE_DIR),
        capture_output=True,  # 출력 캡처
        text=True
    )
    
    # 표준 출력 출력
    if result.stdout:
        print(result.stdout)
    
    # 표준 오류 출력 (경고/오류 메시지)
    if result.stderr:
        print("=== 오류/경고 메시지 ===")
        print(result.stderr)
    
    if result.returncode == 0:
        print("\n[완료] 승선 인원수 탐지가 성공적으로 완료되었습니다!")
    else:
        print(f"\n[오류] 스크립트 실행 중 오류가 발생했습니다. (반환 코드: {result.returncode})")
        print("[INFO] 위의 오류 메시지를 확인하세요.")
else:
    print(f"[ERROR] 스크립트 파일을 찾을 수 없습니다: {script_path}")


[INFO] 스크립트 실행 시작: c:\Users\Brain\Desktop\watercraft-illegal-analytics\src\passenger_counting.py
[INFO] 이 과정은 시간이 걸릴 수 있습니다...



Exception in thread Thread-3 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Brain\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\Brain\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Brain\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 7: illegal multibyte sequence



[완료] 승선 인원수 탐지가 성공적으로 완료되었습니다!


In [8]:
# 결과 이미지 확인
import matplotlib.pyplot as plt
import cv2

RESULT_DIR = BASE_DIR / "results_onboat"

if RESULT_DIR.exists():
    result_files = list(RESULT_DIR.glob("*.jpg")) + list(RESULT_DIR.glob("*.png"))
    if result_files:
        print(f"[INFO] 결과 이미지: {len(result_files)}개")
        print()
        
        # 첫 번째 결과 이미지 표시
        result_path = result_files[0]
        result_img = cv2.imread(str(result_path))
        if result_img is not None:
            result_img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(14, 10))
            plt.imshow(result_img_rgb)
            plt.title(f"승선 인원수 탐지 결과: {result_path.name}")
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            print(f"\n=== 결과 요약 ===")
            print(f"처리된 이미지 수: {len(result_files)}개")
            print(f"결과 저장 위치: {RESULT_DIR}")
        else:
            print("이미지를 로드할 수 없습니다.")
    else:
        print("결과 이미지가 없습니다.")
else:
    print(f"결과 폴더가 없습니다: {RESULT_DIR}")
    print("[INFO] 먼저 위의 스크립트를 실행하세요.")


결과 폴더가 없습니다: c:\Users\Brain\Desktop\watercraft-illegal-analytics\results_onboat
[INFO] 먼저 위의 스크립트를 실행하세요.
